## Indicadores de Salud de la Ciudad de México

In [1]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, readxl, janitor, stringr, writexl, foreign, downloader, tools)

Cargando paquete requerido: pacman



## Tasa de Mortalidad en menores de 5 años de la Ciudad de México

Descargar datos de Defunciones Regsitradas de INEGI, que es la que se toma de acuerdo con Secretaría de Salud

In [56]:
#Definiciones para antes de descargar los datos
# Crear directorio principal
dir_datos <- 'Estadísticas de Defunciones Registradas (EDR)/Microdatos_1'
if (!dir.exists(dir_datos)) {
  dir.create(dir_datos, recursive = TRUE)
}
print(file.exists(dir_datos)) #True = directorio creado

[1] TRUE


In [ ]:
# Función para descargar y procesar los dbf de defunciones
descargar_defunciones_dbf <- function() {
  options(timeout = 90000)
  lista_defun <- list()
  
  # Lista de URLs con sus años o periodos
  urls <- list(
    '2023' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2023/defunciones_base_datos_2023_dbf.zip',
    '2022' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2022/defunciones_base_datos_2022_dbf.zip',
    '2021' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2021/defunciones_base_datos_2021_dbf.zip',
    '2020' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2020/defunciones_base_datos_2020_dbf.zip'
    #'2015_2019' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/datos/defunciones_generales_base_datos_2015_2019_dbf.zip'
  )
  
  for (periodo in names(urls)) {
    url <- urls[[periodo]]
    zip_path <- file.path(dir_datos, basename(url))
    download.file(url, zip_path, mode = "wb")
    
    # Carpeta para extraer
    unzip_dir <- file.path(dir_datos, paste0('Defunciones_', periodo))
    dir.create(unzip_dir, showWarnings = FALSE)
    
    unzip(zip_path, exdir = unzip_dir)
    unlink(zip_path)  # borrar zip
    
    # Buscar archivos .dbf en el unzip_dir
    dbf_files <- list.files(unzip_dir, pattern = '\\.dbf$', full.names = TRUE)
    
    if(length(dbf_files) == 0) {
      warning(paste0('No se encontró archivo DBF para ', periodo))
      next
    }
    
    # Leer cada DBF y guardarlo en lista
    period_lista_defun <- list()
    for(dbf_file in dbf_files) {
      nombre <- file_path_sans_ext(basename(dbf_file))
      df <- tryCatch({
        read.dbf(dbf_file, as.is = TRUE) %>% as_tibble() %>% clean_names()
      }, error = function(e) {
        warning(paste0('Error leyendo DBF: ', dbf_file))
        NULL
      })
      
      if(!is.null(df)) {
        period_lista_defun[[nombre]] <- df
      }
    }
    
    lista_defun[[periodo]] <- period_lista_defun
  }
  
  save(lista_defun, file = file.path(dir_datos, 'defunciones_lista_defun.RData'))
  return(lista_defun)
}


[1] TRUE


In [37]:
# Ejecutar descarga
defunciones <- descargar_defunciones_dbf()

In [ ]:
#Guardar la lista completa para no volver a descargar los datos
saveRDS(defunciones, file = 'Estadísticas de Defunciones Registradas (EDR)/edr_data.rds')

In [40]:
#Cargar la lista para evitar descargar los datos de nuevo
defunciones <- readRDS('Estadísticas de Defunciones Registradas (EDR)/edr_data.rds')

Descargar los datos del Subsistema de Información sobre Nacimientos (SINAC) de la Secretaría de Salud Federal.

*Comparando los nacimientos de la tabla de nacimientos de datos abiertos, con los de los tabulados de Secretaría de Salud, vemos que los datos ya están ajustados con CONAPO*

In [ ]:
#Definiciones para antes de descargar los datos

#Crear lista para guardar los datos de sinac
lista_nacimientos <- list()

options(timeout = 600) #Aumentar el tiempo de espera para la descarga

In [13]:
#iterar por los años de descarga
    for(year in 19:23) {
#Crear la cadena de los años 2020 a 2023
        
        year_str <- paste0('20', year)
        #Cosntruir la url de descarga
        url <- paste0('http://www.dgis.salud.gob.mx/descargas/datosabiertos/nacimientos/sinac_', year_str, '.zip')
        
        #nombre y carpeta del archivo zip
        zip_file <- paste0('SINAC/sinac_', year_str, '.zip')
        extract_dir <- paste0('SINAC/sinac_', year_str)
#Crear carpeta si no existe
        if(!dir.exists('SINAC')) dir.create('SINAC')

#Intentar descargar archivos con trycatch
        tryCatch({
            download(url, destfile = zip_file, mode = 'wb')
            
            #Crear carpeta de extracción si no existe
            if(!file.exists(extract_dir)) dir.create(extract_dir)
                          
            #Descrompimir el archivo zip
            unzip(zip_file, exdir = extract_dir)

             #Buscar los archivos descomprimidos
            files <- list.files(extract_dir, full.names = TRUE)
    
    #Leer el primer archivo CSV encontrado (ajusta si es otro formato)
            csv_file <- files[grep("\\.csv$", files)][1]
    
        if (!is.na(csv_file)) {
        df <- read.csv(csv_file, stringsAsFactors = FALSE) %>% clean_names()
      
        # Guardar el dataframe en la lista
        lista_nacimientos[[year_str]] <- df
        
        # Eliminar el archivo ZIP
        file.remove(zip_file)
        
        message(paste('Año', year_str, 'procesado correctamente.'))
        } else {
        message(paste('No se encontró archivo CSV en', extract_dir))
        }
    }, error = function(e) {
        message(paste('Error al procesar el año', year_str))
    })
    }


Año 2019 procesado correctamente.

Año 2020 procesado correctamente.

Año 2021 procesado correctamente.

Año 2022 procesado correctamente.

Año 2023 procesado correctamente.



In [23]:
#Guardar la lista completa para no volver a descargar los datos
saveRDS(lista_nacimientos, file = 'SINAC/sinac_data.rds')

In [24]:
#Cargar la lista para evitar descargar los datos de nuevo
lista_nacimientos <- readRDS('SINAC/sinac_data.rds')

In [54]:
#Conjunto de nombres para los dataframes de defunciones
nombres_def <- c('2020' = 'defun20','2021' = 'defun21', '2022' = 'DEFUN22', '2023' = 'DEFUN23')

In [ ]:
#Crear dataframe vacío para almacenar los resultados
resultados <- tibble(anio = integer(), 
                     nacimientos = integer(), 
                     defunciones = integer(), 
                     tasa = numeric())

In [ ]:
#Definimos los años para los que vamos a calcular las tasas
anios <- 2019:2023
resultados <- tibble(año = integer(), nacimientos = integer(), defunciones = integer(), tasa = numeric())


In [ ]:
# Iterar sobre los años y calcular tasas
for (a in anios) {
  anio_str <- as.character(a)
  
  if (!is.null(lista_nacimientos[[anio_str]]) && !is.null(defunciones[[anio_str]])) {
    df_nac <- lista_nacimientos[[anio_str]]
    nombre_df <- nombres_def[anio_str]
    df_def <- defunciones[[anio_str]][[nombre_df]]
    
    nac <- df_nac %>%
      filter(entidadresidencia == 9) %>%
      summarise(nacimientos = n()) %>%
      pull(nacimientos)
    
    def <- df_def %>%
      filter(ent_resid == '09', anio_ocur == a, edad >= 1001, edad < 4005) %>%
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    tasa <- def / nac * 1000
    
    resultados <- resultados %>% add_row(anio = a, nacimientos = nac, defunciones = def, tasa = tasa)
  } else {
    message(paste('Datos incompletos para el año', a))
  }
}

print(resultados)

Datos incompletos para el año 2019



# A tibble: 4 × 4
   anio nacimientos defunciones  tasa
  <int>       <int>       <int> <dbl>
1  2020       87005        1266  14.6
2  2021       77476        1062  13.7
3  2022       80892        1159  14.3
4  2023       78032        1152  14.8
